## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [105]:
%%sql 
postgresql:///animal_shelters
with table1 as (
SELECT 
an.animaltype,
an.weight,
(2021)-CAST(RIGHT(birthdate,4) as INT) as age, 
lc.costs as loc_costs
FROM animals as an
INNER JOIN location_costs as lc
ON an.location = lc.location

WHERE an.animalid NOT IN (SELECT sponsorid FROM sponsored_animals)
),

table_costs as (
SELECT CAST(age as INT) as age2, costs 
FROM age_costs
),

t_dog as (
    SELECT 
    animaltype, weight, 
    CASE
    WHEN weight > 30 THEN 'Large'
    WHEN weight <= 10 THEN 'Small'
    ELSE 'Medium'
END AS sz_cal_dog
FROM animals
WHERE animaltype = 'Dog'),

t_cat as (
    SELECT 
    animaltype, weight, 
    CASE
    WHEN weight > 7 THEN 'Large'
    WHEN weight <= 5 THEN 'Small'
    ELSE 'Medium'
END AS sz_cal_cat
FROM animals
WHERE animaltype = 'Cat'),

t_bird as (
    SELECT 
    animaltype, weight, 
    CASE
    WHEN weight > 1.1 THEN 'Large'
    WHEN weight <= 0.7 THEN 'Small'
    ELSE 'Medium'
END AS sz_cal_bird
FROM animals
WHERE animaltype = 'Bird'),

t_union as (
SELECT animaltype,weight,sz_cal_dog as sz_total FROM t_dog
UNION
SELECT animaltype,weight,sz_cal_cat as sz_total FROM t_cat
UNION
SELECT animaltype,weight,sz_cal_bird as sz_total FROM t_bird),

size_t as (
select t_union.animaltype, t_union.sz_total,t_union.weight, sc.costs as size_costs from t_union
INNER JOIN size_costs as sc
ON (t_union.animaltype = sc.animaltype) and (t_union.sz_total = sc.size)),

sub_t as (
select animaltype,weight, loc_costs, agc.costs as age_costs from table1 
INNER JOIN table_costs as agc
ON table1.age = agc.age2),

sub_t_2 as (
select sub_t.animaltype, sz_total as size, 
(loc_costs + sub_t.age_costs + size_t.size_costs) as total
from sub_t
INNER JOIN size_t 
ON (sub_t.animaltype = size_t.animaltype) and (sub_t.weight = size_t.weight)),

sub_final as (
SELECT animaltype, size, sum(total) as total FROM sub_t_2
GROUP BY animaltype, size
order by animaltype, size DESC)

SELECT lower(animaltype) as animaltype, lower(size) as size, total, ROUND(total/(select sum(total) FROM sub_final)*100,2) as percentage 
from sub_final







9 rows affected.


[('bird', 'small', 1615, Decimal('0.05')),
 ('bird', 'medium', 3130, Decimal('0.09')),
 ('bird', 'large', 8120, Decimal('0.23')),
 ('cat', 'small', 518015, Decimal('14.90')),
 ('cat', 'medium', 250575, Decimal('7.21')),
 ('cat', 'large', 439490, Decimal('12.64')),
 ('dog', 'small', 336530, Decimal('9.68')),
 ('dog', 'medium', 941895, Decimal('27.09')),
 ('dog', 'large', 977665, Decimal('28.12'))]